In [15]:
# === Fantasy PPR Scoring with nflreadpy loader (Pandas pipeline) ===
# - Loads each season via nflreadpy (Polars) → converts to pandas
# - Applies existing filters & ESPN-standard PPR scoring
# - Includes historical team normalization and week caps by era
# - Produces multi-season leaders/weekly (players), D/ST, and kicker tables

import numpy as np
import pandas as pd
import nflreadpy as nfl

# ----------------------------------
# Config
# ----------------------------------
ALL_SEASONS = list(range(1999, 2025))  # 1999–2023 in one pass

def fantasy_week_max(season: int) -> int:
    # Common fantasy policy: stop before NFL's final week
    # <=2020: Week 16; >=2021: Week 17
    return 16 if season <= 2020 else 17

# ----------------------------------
# Team normalization (historical)
# ----------------------------------
# Normalize to SEASON-ACCURATE abbreviations:
# - Rams:   STL through 2015, LAR 2016+
# - Raiders: OAK through 2019, LV 2020+
# - Chargers: SD through 2016, LAC 2017+
# Everything else: pass-through
def normalize_team(team: str, season: int) -> str:
    t = (team or "").upper()
    # Rams
    if season <= 2015 and t in {"LAR", "LA", "STL"}:
        return "STL"
    if season >= 2016 and t in {"LAR", "LA", "STL"}:
        return "LAR"
    # Raiders
    if season <= 2019 and t in {"OAK", "LV"}:
        return "OAK"
    if season >= 2020 and t in {"OAK", "LV"}:
        return "LV"
    # Chargers
    if season <= 2016 and t in {"SD", "LAC"}:
        return "SD"
    if season >= 2017 and t in {"SD", "LAC"}:
        return "LAC"
    return t

# ----------------------------------
# Utilities
# ----------------------------------
def base_filter(df: pd.DataFrame, week_min: int, week_max: int) -> pd.DataFrame:
    out = df.query("season_type == 'REG' and @week_min <= week <= @week_max").copy()
    if "play_deleted" in out.columns:
        out = out[out["play_deleted"].fillna(0) != 1]
    if "play_type" in out.columns:
        out = out[out["play_type"] != "no_play"]
    return out

def safe_col(df, name, default=0):
    return df[name].fillna(0) if name in df.columns else 0

# ----------------------------------
# 1) Player PPR (ESPN-standard)
# ----------------------------------
def player_ppr(pbp_reg: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    df = pbp_reg.copy()

    pass_attempt   = (df.get("pass_attempt", df.get("pass", 0)).fillna(0).astype(int) == 1)
    rush_attempt   = (df.get("rush_attempt", df.get("rush", 0)).fillna(0).astype(int) == 1)
    complete_pass  = (df.get("complete_pass", 0).fillna(0).astype(int) == 1)

    pass_role = pass_attempt  & (df["fantasy_player_id"] == df["passer_player_id"])
    rush_role = rush_attempt  & (df["fantasy_player_id"] == df["rusher_player_id"])
    recv_role = complete_pass & (df["fantasy_player_id"] == df["receiver_player_id"])

    # Passing
    pass_fp = np.where(pass_role,
                       safe_col(df, "passing_yards")*0.04 + safe_col(df, "pass_touchdown")*4 - safe_col(df, "interception")*2,
                       0)
    # Rushing
    rush_fp = np.where(rush_role,
                       safe_col(df, "rushing_yards")*0.1 + safe_col(df, "rush_touchdown")*6,
                       0)
    # Receiving (PPR + yards + receiving TDs are flagged by pass_touchdown)
    reception_pts = np.where(recv_role, 1, 0)
    recv_yards_td = np.where(recv_role,
                             safe_col(df, "receiving_yards")*0.1 + safe_col(df, "pass_touchdown")*6,
                             0)

    # 2-pt conversions: credit receiver on pass or rusher on rush (success only)
    two_pt_success = (safe_col(df, "two_point_attempt") == 1) & (df.get("two_point_conv_result", "") == "success")
    two_pt_recv = two_pt_success & (safe_col(df, "pass") == 1) & (df["fantasy_player_id"] == df["receiver_player_id"])
    two_pt_rush = two_pt_success & (safe_col(df, "rush") == 1) & (df["fantasy_player_id"] == df["rusher_player_id"])
    two_pt_fp = np.where(two_pt_recv | two_pt_rush, 2, 0)

    # Fumbles lost
    fum_fp = np.where((safe_col(df, "fumble_lost") == 1) & (df["fantasy_player_id"] == df.get("fumbled_1_player_id")),
                      -2, 0)

    df["fp"] = pass_fp + rush_fp + reception_pts + recv_yards_td + two_pt_fp + fum_fp

    leaders = (df.groupby(["season","fantasy_player_id","fantasy_player_name"], as_index=False)["fp"].sum()
                 .sort_values(["season","fp"], ascending=[True, False]))
    games = (df.loc[df["fp"] != 0]
               .groupby(["season","fantasy_player_id","fantasy_player_name"], as_index=False)["week"].nunique()
               .rename(columns={"week":"games"}))
    leaders = leaders.merge(games, on=["season","fantasy_player_id","fantasy_player_name"], how="left")
    leaders["ppg"] = leaders["fp"] / leaders["games"]

    weekly = (df.groupby(["season","fantasy_player_id","fantasy_player_name","week"], as_index=False)["fp"].sum()
                .sort_values(["season","fantasy_player_name","week"]))

    return leaders, weekly

# ----------------------------------
# 2) Defense/Special Teams (D/ST)
# ----------------------------------
def dst_scoring(pbp_reg: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    df = pbp_reg.copy()

    # Final scores + teams per game
    scores = (df.groupby(["season","game_id"], as_index=False)[["total_home_score","total_away_score","home_team","away_team"]]
                .agg({"total_home_score":"max","total_away_score":"max","home_team":"first","away_team":"first"})
                .rename(columns={"total_home_score":"home_pts_final","total_away_score":"away_pts_final"}))

    home_rows = scores[["season","game_id","home_team","away_pts_final"]].rename(
        columns={"home_team":"team","away_pts_final":"points_allowed"})
    away_rows = scores[["season","game_id","away_team","home_pts_final"]].rename(
        columns={"away_team":"team","home_pts_final":"points_allowed"})
    pa_tbl = pd.concat([home_rows, away_rows], ignore_index=True)

    # Points-allowed tiers (ESPN-ish)
    def pa_to_pts(pa: int) -> int:
        if pa == 0: return 10
        if pa <= 6: return 7
        if pa <= 13: return 4
        if pa <= 17: return 1
        if pa <= 27: return 0
        if pa <= 34: return -1
        if pa <= 45: return -4
        return -5

    pa_tbl["pa_pts"] = pa_tbl["points_allowed"].astype(int).map(pa_to_pts)

    # Tallies by defense
    by_def = (df.groupby(["season","game_id","defteam"], as_index=False)
                .agg({"sack":"sum","interception":"sum","safety":"sum","punt_blocked":"sum"})
                .rename(columns={"defteam":"team"}))

    # Blocked FGs
    tmp = df.copy()
    tmp["fg_block"] = (tmp.get("field_goal_result","").astype(str).str.lower() == "blocked").astype(int)
    by_def_fgblk = (tmp.groupby(["season","game_id","defteam"], as_index=False)["fg_block"].sum()
                      .rename(columns={"defteam":"team"}))

    # Fumble recoveries credited to recovering team
    fr_parts = []
    if "fumble_recovery_1_team" in df.columns:
        fr1 = df[df["fumble_recovery_1_team"].notna()][["season","game_id","fumble_recovery_1_team"]].copy()
        fr1["team"] = fr1["fumble_recovery_1_team"]; fr1["fr"] = 1
        fr_parts.append(fr1[["season","game_id","team","fr"]])
    if "fumble_recovery_2_team" in df.columns:
        fr2 = df[df["fumble_recovery_2_team"].notna()][["season","game_id","fumble_recovery_2_team"]].copy()
        fr2["team"] = fr2["fumble_recovery_2_team"]; fr2["fr"] = 1
        fr_parts.append(fr2[["season","game_id","team","fr"]])
    by_fr = (pd.concat(fr_parts, ignore_index=True)
             .groupby(["season","game_id","team"], as_index=False)["fr"].sum()) if fr_parts else \
            pd.DataFrame(columns=["season","game_id","team","fr"])

    # Return TDs (credited to return team)
    ret = df[(safe_col(df,"return_touchdown") == 1)]
    by_ret_td = (ret.groupby(["season","game_id","return_team"], as_index=False).size()
                   .rename(columns={"return_team":"team","size":"ret_td"}))

    # Merge all components
    dst = pa_tbl[["season","game_id","team","pa_pts"]].copy()
    for part in [
        by_def[["season","game_id","team","sack","interception","safety","punt_blocked"]],
        by_def_fgblk[["season","game_id","team","fg_block"]],
        by_fr[["season","game_id","team","fr"]],
        by_ret_td[["season","game_id","team","ret_td"]],
    ]:
        dst = dst.merge(part, on=["season","game_id","team"], how="left")

    dst = dst.fillna(0)
    for c in ["sack","interception","safety","punt_blocked","fg_block","fr","ret_td"]:
        if c in dst.columns: dst[c] = dst[c].astype(int)

    # Normalize historical team code
    dst["team"] = [normalize_team(t, s) for t, s in zip(dst["team"].astype(str), dst["season"].astype(int))]

    dst["dst_fp"] = (
        dst["sack"]*1 + dst["interception"]*2 + dst["fr"]*2 + dst["safety"]*2
        + dst["ret_td"]*6 + (dst["punt_blocked"] + dst["fg_block"])*2 + dst["pa_pts"]
    )

    per_game = dst.sort_values(["season","game_id","team"]).reset_index(drop=True)
    season = (per_game.groupby(["season","team"], as_index=False)["dst_fp"].sum()
                      .sort_values(["season","dst_fp"], ascending=[True, False]))
    return season, per_game

# ----------------------------------
# 3) Kickers (ESPN-style)
# ----------------------------------
def kicker_scoring(pbp_reg: pd.DataFrame, miss_pat_minus_one=False) -> tuple[pd.DataFrame, pd.DataFrame]:
    df = pbp_reg.copy()
    kick = df[(safe_col(df, "field_goal_attempt") == 1) | (safe_col(df, "extra_point_attempt") == 1)].copy()

    for c in ["field_goal_result","extra_point_result"]:
        if c in kick.columns: 
            kick[c] = kick[c].astype(str).str.lower()

    # FG made by distance
    fg_made = (safe_col(kick, "field_goal_attempt") == 1) & (kick["field_goal_result"] == "made")
    dist = safe_col(kick, "kick_distance").astype(float)
    fg_pts = np.where(fg_made & (dist >= 60), 6,
              np.where(fg_made & (dist >= 50), 5,
              np.where(fg_made & (dist >= 40), 4,
              np.where(fg_made, 3, 0))))

    # Missed FG (includes blocked)
    fg_miss = (safe_col(kick, "field_goal_attempt") == 1) & (kick["field_goal_result"].isin(["missed","blocked"]))
    fg_miss_pts = np.where(fg_miss, -1, 0)

    # PATs
    pat_attempt = (safe_col(kick, "extra_point_attempt") == 1)
    pat_good = pat_attempt & (kick["extra_point_result"].isin(["good","made"]))
    pat_pts = np.where(pat_good, 1, 0)
    pat_missed = pat_attempt & (kick["extra_point_result"].isin(["failed","missed","blocked"]))
    pat_miss_pts = np.where(miss_pat_minus_one & pat_missed, -1, 0)

    kick["k_fp"] = fg_pts + fg_miss_pts + pat_pts + pat_miss_pts

    id_col = "kicker_player_id" if "kicker_player_id" in kick.columns else "fantasy_player_id"
    name_col = "kicker_player_name" if "kicker_player_name" in kick.columns else "fantasy_player_name"

    per_game = (kick.groupby(["season","game_id", id_col, name_col, "posteam"], as_index=False)
                  .agg(k_fp=("k_fp","sum"),
                       fg_made=("field_goal_attempt", lambda s: int(((s==1) & (kick.loc[s.index,'field_goal_result'].eq('made'))).sum())),
                       fg_att=("field_goal_attempt","sum"),
                       pat_made=("extra_point_attempt", lambda s: int(((s==1) & (kick.loc[s.index,'extra_point_result'].isin(['good','made']))).sum())),
                       pat_att=("extra_point_attempt","sum"))
               )

    # Normalize team code historically for posteam in outputs
    per_game["posteam"] = [normalize_team(t, s) for t, s in zip(per_game["posteam"].astype(str), per_game["season"].astype(int))]

    season = (per_game.groupby(["season", id_col, name_col, "posteam"], as_index=False)
                      .agg(total_fp=("k_fp","sum"),
                           games=("game_id","nunique"),
                           fg_made=("fg_made","sum"),
                           fg_att=("fg_att","sum"),
                           pat_made=("pat_made","sum"),
                           pat_att=("pat_att","sum")))
    season["ppg"] = season["total_fp"] / season["games"]
    season = season.sort_values(["season","total_fp","ppg"], ascending=[True, False, False])
    return season, per_game

# ----------------------------------
# Runner (streams seasons safely) — uses nflreadpy loader
# ----------------------------------
players_all, players_weekly_all = [], []
dst_all, dst_games_all = [], []
k_all, k_games_all = [], []

for yr in ALL_SEASONS:
    print(f"Processing {yr} ...")
    # Load via nflreadpy (Polars) then convert to pandas
    pbp_pl = nfl.load_pbp([yr])      # Polars DataFrame
    pbp_y  = pbp_pl.to_pandas()      # hand off to pandas pipeline

    wk_max = fantasy_week_max(yr)
    pbp_reg = base_filter(pbp_y, week_min=1, week_max=wk_max)

    # Optional RAM saver
    for c in pbp_reg.select_dtypes("float64").columns:
        pbp_reg[c] = pbp_reg[c].astype("float32")
    print(f"{yr} done.\nDowncasting floats.")

    # Players
    p_season, p_weekly = player_ppr(pbp_reg)
    players_all.append(p_season)
    players_weekly_all.append(p_weekly)

    # D/ST
    d_season, d_games = dst_scoring(pbp_reg)
    dst_all.append(d_season)
    dst_games_all.append(d_games)

    # Kickers
    k_season, k_games = kicker_scoring(pbp_reg, miss_pat_minus_one=False)
    k_all.append(k_season)
    k_games_all.append(k_games)

# Concatenate multi-season outputs
players_leaders_multi = pd.concat(players_all, ignore_index=True)
players_weekly_multi  = pd.concat(players_weekly_all, ignore_index=True)
dst_season_multi      = pd.concat(dst_all, ignore_index=True)
dst_per_game_multi    = pd.concat(dst_games_all, ignore_index=True)
k_season_multi        = pd.concat(k_all, ignore_index=True)
k_per_game_multi      = pd.concat(k_games_all, ignore_index=True)

# Examples:
display(players_leaders_multi.groupby("season").head(10))
display(dst_season_multi.groupby("season").head(10))
display(k_season_multi.groupby("season").head(10))

# (Optional) Save to files for dashboards
# players_leaders_multi.to_parquet("players_ppr_allseasons.parquet", index=False)
# players_weekly_multi.to_parquet("players_weekly_ppr_allseasons.parquet", index=False)
# dst_season_multi.to_parquet("dst_season_allseasons.parquet", index=False)
# dst_per_game_multi.to_parquet("dst_per_game_allseasons.parquet", index=False)
# k_season_multi.to_parquet("kickers_season_allseasons.parquet", index=False)
# k_per_game_multi.to_parquet("kickers_per_game_allseasons.parquet", index=False)

Processing 1999 ...
1999 done.
Downcasting floats.
Processing 2000 ...
2000 done.
Downcasting floats.
Processing 2001 ...
2001 done.
Downcasting floats.
Processing 2002 ...
2002 done.
Downcasting floats.
Processing 2003 ...
2003 done.
Downcasting floats.
Processing 2004 ...
2004 done.
Downcasting floats.
Processing 2005 ...
2005 done.
Downcasting floats.
Processing 2006 ...
2006 done.
Downcasting floats.
Processing 2007 ...
2007 done.
Downcasting floats.
Processing 2008 ...
2008 done.
Downcasting floats.
Processing 2009 ...
2009 done.
Downcasting floats.
Processing 2010 ...
2010 done.
Downcasting floats.
Processing 2011 ...
2011 done.
Downcasting floats.
Processing 2012 ...
2012 done.
Downcasting floats.
Processing 2013 ...
2013 done.
Downcasting floats.
Processing 2014 ...
2014 done.
Downcasting floats.
Processing 2015 ...
2015 done.
Downcasting floats.
Processing 2016 ...
2016 done.
Downcasting floats.
Processing 2017 ...
2017 done.
Downcasting floats.
Processing 2018 ...
2018 done.


C:\Users\macsa\anaconda3\Lib\site-packages\IPython\core\formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,season,fantasy_player_id,fantasy_player_name,fp,games,ppg
0,1999,00-0008241,E.James,357.600002,15.0,23.840000
1,1999,00-0005092,M.Faulk,354.700004,14.0,25.335715
2,1999,00-0007024,M.Harrison,344.600005,15.0,22.973334
3,1999,00-0005883,E.George,287.800003,15.0,19.186667
4,1999,00-0015218,J.Smith,285.100003,15.0,19.006667
...,...,...,...,...,...,...
13979,2024,00-0039139,J.Gibbs,306.800002,16.0,19.175000
13980,2024,00-0032764,D.Henry,306.300003,16.0,19.143750
13981,2024,00-0039040,D.Achane,281.800002,16.0,17.612500
13982,2024,00-0035700,J.Jacobs,281.500002,16.0,17.593750


C:\Users\macsa\anaconda3\Lib\site-packages\IPython\core\formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,season,team,dst_fp
0,1999,STL,236
1,1999,JAX,212
2,1999,KC,197
3,1999,DAL,190
4,1999,BAL,181
...,...,...,...
802,2024,SEA,138
803,2024,BUF,136
804,2024,PHI,136
805,2024,DAL,127


C:\Users\macsa\anaconda3\Lib\site-packages\IPython\core\formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,season,kicker_player_id,kicker_player_name,posteam,total_fp,games,fg_made,fg_att,pat_made,pat_att,ppg
0,1999,00-0010373,O.Mare,MIA,148,15,38,45.0,26,26.0,9.866667
1,1999,00-0016830,M.Vanderjagt,IND,147,15,32,36.0,43,43.0,9.800000
2,1999,00-0012875,T.Peterson,SEA,133,15,31,37.0,32,32.0,8.866667
3,1999,00-0007622,M.Hollis,JAX,131,15,30,35.0,34,34.0,8.733333
4,1999,00-0004811,J.Elam,DEN,119,15,27,34.0,29,29.0,7.933333
...,...,...,...,...,...,...,...,...,...,...,...
1083,2024,00-0035358,C.McLaughlin,TB,151,16,28,30.0,51,53.0,9.437500
1084,2024,00-0039172,J.Bates,DET,150,16,25,28.0,60,63.0,9.375000
1085,2024,00-0032569,W.Lutz,DEN,149,16,30,33.0,41,41.0,9.312500
1086,2024,00-0036162,T.Bass,BUF,137,16,23,28.0,58,62.0,8.562500


In [16]:
# --- Build (season, week, player_id) -> team, position lookup and merge into players_weekly_multi ---

import pandas as pd
import numpy as np
import nflreadpy as nfl

# 1) Minimal per-play load to infer TEAM per player-week (mode of posteam where player was involved)
def build_player_week_team(seasons):
    parts = []
    for yr in seasons:
        print(f"[team map] loading {yr} ...")
        df = nfl.load_pbp([yr]).to_pandas()
        # keep light
        keep = ["season","week","posteam","fantasy_player_id",
                "pass_attempt","rush_attempt","complete_pass",
                "passer_player_id","rusher_player_id","receiver_player_id"]
        keep = [c for c in keep if c in df.columns]
        df = df[keep].copy()
        df.columns = [c.lower() for c in df.columns]

        # where did the fantasy player "act" on the play?
        pass_attempt = (df.get("pass_attempt", 0).fillna(0).astype(int) == 1)
        rush_attempt = (df.get("rush_attempt", 0).fillna(0).astype(int) == 1)
        complete_pass = (df.get("complete_pass", 0).fillna(0).astype(int) == 1)

        is_actor = (
            (pass_attempt & (df["fantasy_player_id"] == df.get("passer_player_id"))) |
            (rush_attempt & (df["fantasy_player_id"] == df.get("rusher_player_id"))) |
            (complete_pass & (df["fantasy_player_id"] == df.get("receiver_player_id")))
        )

        act = df.loc[is_actor, ["season","week","posteam","fantasy_player_id"]].dropna(subset=["fantasy_player_id"])
        if "posteam" not in act.columns:
            act["posteam"] = np.nan

        # team per (season,week,player): mode of posteam
        team_mode = (act.groupby(["season","week","fantasy_player_id","posteam"], as_index=False)
                        .size()
                        .sort_values(["season","week","fantasy_player_id","size"], ascending=[True,True,True,False]))
        team_mode = team_mode.drop_duplicates(["season","week","fantasy_player_id"]).rename(columns={"fantasy_player_id":"player_id","posteam":"team"})
        parts.append(team_mode[["season","week","player_id","team"]])
    team_map = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["season","week","player_id","team"])
    return team_map

# 2) Position map — prefer roster if available; fallback to role inference per season
def build_player_position_map(seasons):
    pos_parts = []
    for yr in seasons:
        print(f"[pos map] loading {yr} ...")
        df = nfl.load_pbp([yr]).to_pandas()
        keep = ["season","fantasy_player_id",
                "pass_attempt","rush_attempt","complete_pass",
                "passer_player_id","rusher_player_id","receiver_player_id"]
        keep = [c for c in keep if c in df.columns]
        df = df[keep].copy()
        df.columns = [c.lower() for c in df.columns]

        # counts by role per player-season
        df["is_pass"] = ((df.get("pass_attempt",0)==1) & (df["fantasy_player_id"] == df.get("passer_player_id"))).astype(int)
        df["is_rush"] = ((df.get("rush_attempt",0)==1) & (df["fantasy_player_id"] == df.get("rusher_player_id"))).astype(int)
        df["is_recv"] = ((df.get("complete_pass",0)==1) & (df["fantasy_player_id"] == df.get("receiver_player_id"))).astype(int)

        grp = (df.groupby(["season","fantasy_player_id"], as_index=False)[["is_pass","is_rush","is_recv"]].sum()
                 .rename(columns={"fantasy_player_id":"player_id"}))

        # heuristics if roster not used:
        # mostly passer => QB; mostly rush > recv => RB; else pass-catcher => WR (could be TE; you can refine with a roster later)
        def infer_pos(row):
            if row["is_pass"] >= max(row["is_rush"], row["is_recv"]): return "QB"
            if row["is_rush"] > row["is_recv"]: return "RB"
            return "WR"  # WR/TE bucket; refine with roster if needed

        grp["position"] = grp.apply(infer_pos, axis=1)
        pos_parts.append(grp[["season","player_id","position"]])
    pos_map = pd.concat(pos_parts, ignore_index=True) if pos_parts else pd.DataFrame(columns=["season","player_id","position"])
    return pos_map

# Build maps just for seasons present in your weekly table
_seasons = sorted(players_weekly_multi["season"].unique().tolist())
team_map = build_player_week_team(_seasons)
pos_map  = build_player_position_map(_seasons)

# Merge maps into players_weekly_multi
pw = players_weekly_multi.copy().rename(columns={
    "fantasy_player_id":"player_id",
    "fantasy_player_name":"player_name"
})
pw = pw.merge(team_map, on=["season","week","player_id"], how="left")
pw = pw.merge(pos_map,  on=["season","player_id"],            how="left")

# Final sanity: fill missing team/position if any
pw["team"] = pw["team"].fillna("UNK")
pw["position"] = pw["position"].fillna("WR")  # neutral default for pass-catcher

# Expose pw for downstream feature builder
players_weekly_with_ctx = pw

print("players_weekly_with_ctx columns:", players_weekly_with_ctx.columns.tolist()[:25], "...")
print("Sample:\n", players_weekly_with_ctx.head(8).to_string(index=False))


[team map] loading 1999 ...
[team map] loading 2000 ...
[team map] loading 2001 ...
[team map] loading 2002 ...
[team map] loading 2003 ...
[team map] loading 2004 ...
[team map] loading 2005 ...
[team map] loading 2006 ...
[team map] loading 2007 ...
[team map] loading 2008 ...
[team map] loading 2009 ...
[team map] loading 2010 ...
[team map] loading 2011 ...
[team map] loading 2012 ...
[team map] loading 2013 ...
[team map] loading 2014 ...
[team map] loading 2015 ...
[team map] loading 2016 ...
[team map] loading 2017 ...
[team map] loading 2018 ...
[team map] loading 2019 ...
[team map] loading 2020 ...
[team map] loading 2021 ...
[team map] loading 2022 ...
[team map] loading 2023 ...
[team map] loading 2024 ...
[pos map] loading 1999 ...
[pos map] loading 2000 ...
[pos map] loading 2001 ...
[pos map] loading 2002 ...
[pos map] loading 2003 ...
[pos map] loading 2004 ...
[pos map] loading 2005 ...
[pos map] loading 2006 ...
[pos map] loading 2007 ...
[pos map] loading 2008 ...
[p

In [17]:
import pandas as pd
import numpy as np

# ---- helpers ----
def exists(df, col): 
    return col in df.columns

def add_team_week_aggregates(pwk: pd.DataFrame):
    """Compute team totals per season-week to enable shares, then merge back."""
    need = ["targets","receptions","carries","receiving_yards","rushing_yards","air_yards","fantasy_player_id"]
    have = [c for c in need if exists(pwk, c)]
    base = pwk[["season","week","team"] + have].copy()
    team_tot = base.groupby(["season","week","team"], as_index=False).sum()
    rename = {c: f"team_{c}" for c in have if c != "fantasy_player_id"}
    team_tot = team_tot.rename(columns=rename).drop(columns=[c for c in team_tot.columns if c.endswith("fantasy_player_id")], errors="ignore")
    return pwk.merge(team_tot, on=["season","week","team"], how="left")

def add_shifted_rolls(df, by, cols, windows=(3,), prefix="r"):
    """For each col, add shifted rolling means/sums grouped by keys in `by`."""
    df = df.sort_values(by + ["season","week"])
    for c in cols:
        if not exists(df, c): 
            continue
        for w in windows:
            roll = (df.groupby(by)[c]
                      .shift(1)  # avoid leakage
                      .rolling(w, min_periods=1)
                      .mean())
            df[f"{prefix}_{c}_{w}"] = roll.values
    return df

def season_splits(df, season_col="season", test_season=2024, val_season=2023):
    tr = df[df[season_col] < val_season].copy()
    va = df[df[season_col] == val_season].copy()
    te = df[df[season_col] == test_season].copy()
    return tr, va, te

# ---- Skill positions table ----
pw = players_weekly_with_ctx.copy()

# unify column names we rely on
rename_map = {
    "fantasy_player_id":"player_id",
    "fantasy_player_name":"player_name"
}
pw = pw.rename(columns={k:v for k,v in rename_map.items() if k in pw.columns})

# ensure required keys
need_keys = ["season","week","player_id","player_name"]
for k in need_keys:
    if k not in pw.columns:
        raise ValueError(f"Missing required column in players_weekly_multi: {k}")

# team normalization already handled earlier; assume 'team' exists downstream. 
if "team" not in pw.columns and "posteam" in pw.columns:
    pw["team"] = pw["posteam"]

# basic feature cols (keep if present)
base_feats = [
    "targets","receptions","carries",
    "receiving_yards","rushing_yards",
    "passing_yards","passing_tds","rushing_tds","receiving_tds","interceptions",
    "fumbles_lost","air_yards"
]
keep = [c for c in base_feats if c in pw.columns]
fe = pw[["season","week","player_id","player_name","fp","team","position"] + keep].copy()

# shares (need team totals)
fe = add_team_week_aggregates(fe)
if "targets" in fe.columns and "team_targets" in fe.columns:
    fe["target_share"] = (fe["targets"] / fe["team_targets"]).fillna(0)
if "carries" in fe.columns and "team_carries" in fe.columns:
    fe["carry_share"] = (fe["carries"] / fe["team_carries"]).fillna(0)
if "air_yards" in fe.columns and "team_air_yards" in fe.columns:
    fe["air_yards_share"] = (fe["air_yards"] / fe["team_air_yards"]).fillna(0)

# rolling player form/features (3,5,8 games)
roll_cols = [c for c in ["fp","targets","receptions","carries","receiving_yards","rushing_yards","passing_yards"] if c in fe.columns]
fe = add_shifted_rolls(fe, by=["player_id"], cols=roll_cols, windows=(3,5,8), prefix="r")

# simple team form (rolling mean of total team points per week using players table as proxy)
# If you have per-game final scores elsewhere, prefer that.
if "fp" in fe.columns:
    team_week_pts = fe.groupby(["season","week","team"], as_index=False)["fp"].sum().rename(columns={"fp":"team_fp_sum"})
    fe = fe.merge(team_week_pts, on=["season","week","team"], how="left")
    fe = add_shifted_rolls(fe, by=["team"], cols=["team_fp_sum"], windows=(3,5), prefix="r")

# drop rows with missing target
fe = fe.dropna(subset=["fp"]).reset_index(drop=True)

# split
train_df, val_df, test_df = season_splits(fe, test_season=2024, val_season=2023)

print(f"Skill positions — shapes: train={train_df.shape}, val={val_df.shape}, test={test_df.shape}")


Skill positions — shapes: train=(107833, 13), val=(5000, 13), test=(4935, 13)


In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# --- quick diagnostics: which columns have NaNs?
def null_report(df, cols):
    rep = pd.Series({c: df[c].isna().sum() for c in cols if c in df.columns})
    rep = rep[rep > 0].sort_values(ascending=False)
    if len(rep):
        print("Columns with NaNs (top 20):")
        print(rep.head(20))
    else:
        print("No NaNs detected in selected columns.")

# choose columns actually present
num_cols = [c for c in [
    "targets","receptions","carries",
    "receiving_yards","rushing_yards",
    "passing_yards","passing_tds","rushing_tds","receiving_tds","interceptions","fumbles_lost","air_yards",
    "target_share","carry_share","air_yards_share",
    "r_fp_3","r_fp_5","r_fp_8",
    "r_targets_3","r_receptions_3","r_carries_3",
    "r_receiving_yards_3","r_rushing_yards_3","r_passing_yards_3",
    "r_team_fp_sum_3","r_team_fp_sum_5"
] if c in train_df.columns]

cat_cols = [c for c in ["team","position"] if c in train_df.columns]

# bail out early if empty feature sets
if not num_cols and not cat_cols:
    raise ValueError("No features found in train_df. Check earlier feature-building steps.")

Xtr = train_df[num_cols + cat_cols].copy()
ytr = train_df["fp"].values
Xva = val_df[num_cols + cat_cols].copy()
yva = val_df["fp"].values
Xte = test_df[num_cols + cat_cols].copy()
yte = test_df["fp"].values

print(f"Feature counts — numeric: {len(num_cols)}, categorical: {len(cat_cols)}")
null_report(train_df, num_cols + cat_cols)

# --- preprocessing with imputers ---
numeric_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])
categorical_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

pre = ColumnTransformer([
    ("num", numeric_pipe, [c for c in num_cols if c in Xtr.columns]),
    ("cat", categorical_pipe, [c for c in cat_cols if c in Xtr.columns]),
])

def eval_model(pipe, name):
    pipe.fit(Xtr, ytr)
    for split, X, y in [("VAL", Xva, yva), ("TEST", Xte, yte)]:
        yhat = pipe.predict(X)
        mae = mean_absolute_error(y, yhat)
        rmse = mean_squared_error(y, yhat, squared=False)
        r2 = r2_score(y, yhat)
        print(f"{name} [{split}]  MAE={mae:.3f}  RMSE={rmse:.3f}  R2={r2:.3f}")
    print("-"*60)

lin = Pipeline([("prep", pre), ("reg", LinearRegression())])
eval_model(lin, "Linear")

ridge = Pipeline([("prep", pre), ("reg", Ridge(alpha=3.0, random_state=42))])
eval_model(ridge, "Ridge")

rf = Pipeline([("prep", pre), ("reg", RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))])
eval_model(rf, "RandomForest")


Feature counts — numeric: 5, categorical: 2
Columns with NaNs (top 20):
r_fp_3             88
r_fp_5              2
r_fp_8              1
r_team_fp_sum_3     1
r_team_fp_sum_5     1
dtype: int64


C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear [VAL]  MAE=4.184  RMSE=5.698  R2=0.390
Linear [TEST]  MAE=4.263  RMSE=5.742  R2=0.401
------------------------------------------------------------


C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ridge [VAL]  MAE=4.184  RMSE=5.698  R2=0.390
Ridge [TEST]  MAE=4.263  RMSE=5.741  R2=0.401
------------------------------------------------------------


KeyboardInterrupt: 

In [20]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr

def eval_by_position(model, X, y, df_meta, pos_col="position", label="TEST"):
    yhat = model.predict(X)
    out = []
    for p, g_idx in df_meta.groupby(pos_col).groups.items():
        g_y = y[g_idx]; g_yhat = yhat[g_idx]
        if len(g_y) < 20: 
            continue
        out.append({
            "split": label,
            "position": p,
            "MAE": mean_absolute_error(g_y, g_yhat),
            "RMSE": mean_squared_error(g_y, g_yhat, squared=False),
            "R2": r2_score(g_y, g_yhat),
            "Spearman": spearmanr(g_y, g_yhat, nan_policy="omit").correlation
        })
    return pd.DataFrame(out)

# assuming `ridge` is fitted from your last cell, and Xte/yte come from earlier
bypos = eval_by_position(ridge, Xte, yte, test_df.reset_index(drop=True))
display(bypos.sort_values("MAE"))


C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\macsa\AppData\Local\Temp\ipykernel_5532\193476969.py:18: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "Spearman": spearmanr(g_y, g_yhat, nan_policy="omit").correlation
C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.


,split,position,MAE,RMSE,R2,Spearman
0,TEST,QB,1.323799,1.641513,0.000000,NaN
2,TEST,WR,4.258850,5.755111,0.379485,0.669390
1,TEST,RB,4.307955,5.753037,0.425035,0.650492


In [21]:
from sklearn.model_selection import GridSearchCV

ridge_grid = GridSearchCV(
    estimator=ridge, 
    param_grid={"reg__alpha":[0.1, 0.3, 1.0, 3.0, 10.0]},
    scoring="neg_mean_absolute_error",
    cv=3, n_jobs=-1
).fit(Xtr, ytr)

rf_grid = GridSearchCV(
    estimator=rf,
    param_grid={"reg__max_depth":[None, 8, 14], "reg__min_samples_leaf":[1, 3, 7]},
    scoring="neg_mean_absolute_error",
    cv=3, n_jobs=-1
).fit(Xtr, ytr)

print("Ridge best:", ridge_grid.best_params_)
print("RF best:", rf_grid.best_params_)


Ridge best: {'reg__alpha': 10.0}
RF best: {'reg__max_depth': 14, 'reg__min_samples_leaf': 7}


In [23]:
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np

class MixedMoE(BaseEstimator, RegressorMixin):
    def __init__(self, preprocessor, experts_by_pos, pos_col="position"):
        self.pre_template = preprocessor         # template to clone
        self.experts_by_pos = experts_by_pos     # dict: pos -> estimator
        self.pos_col = pos_col
        self.gate = None
        self.classes_ = None
        self.experts_ = {}

    def fit(self, X, y):
        # Clone preprocessor for the gating model (don't reuse!)
        gate_pre = clone(self.pre_template)
        self.gate = Pipeline([
            ("prep", gate_pre),
            ("clf", LogisticRegression(max_iter=2000, multi_class="multinomial"))
        ])
        self.gate.fit(X, X[self.pos_col])
        self.classes_ = list(self.gate.named_steps["clf"].classes_)

        # Fit each expert on its slice, each with its own cloned preprocessor
        self.experts_.clear()
        for pos, base_model in self.experts_by_pos.items():
            idx = (X[self.pos_col] == pos).values
            if idx.sum() == 0:
                continue
            model = Pipeline([
                ("prep", clone(self.pre_template)),
                ("reg", clone(base_model))
            ])
            model.fit(X.iloc[idx], y[idx])
            self.experts_[pos] = model
        return self

    def predict(self, X):
        proba = self.gate.predict_proba(X)  # uses its own fitted pre
        yhat = np.zeros(len(X))
        for i, cls in enumerate(self.classes_):
            if cls in self.experts_:
                yhat += proba[:, i] * self.experts_[cls].predict(X)  # expert has its own fitted pre
        return yhat

# Use your tuned params
ridge_alpha = ridge_grid.best_params_.get("reg__alpha", 10.0) if 'ridge_grid' in globals() else 10.0
rf_depth    = rf_grid.best_params_.get("reg__max_depth", 14)    if 'rf_grid' in globals() else 14
rf_leaf     = rf_grid.best_params_.get("reg__min_samples_leaf", 7) if 'rf_grid' in globals() else 7

experts = {
    "QB": Ridge(alpha=ridge_alpha, random_state=42),
    "RB": RandomForestRegressor(n_estimators=400, max_depth=rf_depth, min_samples_leaf=rf_leaf, random_state=42, n_jobs=-1),
    "WR": RandomForestRegressor(n_estimators=400, max_depth=rf_depth, min_samples_leaf=rf_leaf, random_state=42, n_jobs=-1),
    "TE": RandomForestRegressor(n_estimators=400, max_depth=rf_depth, min_samples_leaf=rf_leaf, random_state=42, n_jobs=-1),
}

moe = MixedMoE(preprocessor=pre, experts_by_pos=experts)
moe.fit(Xtr, ytr)

for tag, X, y in [("VAL", Xva, yva), ("TEST", Xte, yte)]:
    pred = moe.predict(X)
    print(f"MoE [{tag}]  MAE={mean_absolute_error(y,pred):.3f}  RMSE={mean_squared_error(y,pred, squared=False):.3f}  R2={r2_score(y,pred):.3f}")


C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


MoE [VAL]  MAE=4.006  RMSE=5.606  R2=0.409
MoE [TEST]  MAE=4.099  RMSE=5.689  R2=0.412


C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [25]:
import numpy as np
import json, os
from sklearn.base import clone
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1) Force-align columns for VAL/TEST to training feature set
Xva = Xva.reindex(columns=Xtr.columns, fill_value=np.nan)
Xte = Xte.reindex(columns=Xtr.columns, fill_value=np.nan)

# 2) Rebuild best-tuned pipelines (fresh fit) so they exactly match current X*
ridge_best = clone(ridge)   # ridge is your pipeline with pre+Ridge
ridge_best.set_params(**{"reg__alpha": ridge_grid.best_params_.get("reg__alpha", 10.0)})
ridge_best.fit(Xtr, ytr)

rf_best = clone(rf)         # rf is your pipeline with pre+RF
rf_best.set_params(**{
    "reg__max_depth": rf_grid.best_params_.get("reg__max_depth", 14),
    "reg__min_samples_leaf": rf_grid.best_params_.get("reg__min_samples_leaf", 7),
})
rf_best.fit(Xtr, ytr)

# (Optional) Refit MoE too, in case columns changed upstream
moe.fit(Xtr, ytr)

# 3) Utility: RMSE function without the deprecation warning
try:
    from sklearn.metrics import root_mean_squared_error as rmse_func
except Exception:
    rmse_func = lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False)

def evaluate_and_save(model, name, X, y, split):
    yhat = model.predict(X)
    metrics = {
        "MAE": float(mean_absolute_error(y, yhat)),
        "RMSE": float(rmse_func(y, yhat)),
        "R2": float(r2_score(y, yhat))
    }
    os.makedirs("outputs", exist_ok=True)
    with open(f"outputs/metrics_{name}_{split}.json","w") as f:
        json.dump(metrics, f, indent=2)
    print(f"{name} [{split}]  MAE={metrics['MAE']:.3f}  RMSE={metrics['RMSE']:.3f}  R2={metrics['R2']:.3f}")

# 4) Save metrics for all three models on VAL and TEST
for split, X, y in [("val", Xva, yva), ("test", Xte, yte)]:
    evaluate_and_save(ridge_best, "ridge", X, y, split)
    evaluate_and_save(rf_best,    "rf",    X, y, split)
    evaluate_and_save(moe,        "moe",   X, y, split)

# 5) (Optional) Persist the exact feature set used for training (helps teammates reproduce)
with open("outputs/feature_columns.json","w") as f:
    json.dump(list(Xtr.columns), f, indent=2)
print("Saved outputs/feature_columns.json")


C:\Users\macsa\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


ridge [val]  MAE=4.184  RMSE=5.697  R2=0.390
rf [val]  MAE=3.992  RMSE=5.582  R2=0.414
moe [val]  MAE=4.006  RMSE=5.606  R2=0.409
ridge [test]  MAE=4.263  RMSE=5.741  R2=0.401
rf [test]  MAE=4.091  RMSE=5.665  R2=0.417
moe [test]  MAE=4.099  RMSE=5.689  R2=0.412
Saved outputs/feature_columns.json
